In [1]:
import matplotlib.pyplot as plt
def get_academic_colors():
    """返回您指定的学术风格色板"""
    return {
        'primary': '#333333',
        'secondary': '#000000',
        'background': '#F0F0F0',
        'gridline': '#D9D9D9',
        'categorical': [
            '#FF8C00',  # 明亮橙色
            '#6A7FDB',  # 明亮靛蓝
            '#52BCA3',  # 浅青绿
            '#A0522D',  # 深红棕色
            '#DA70D6',  # 明亮兰花紫
            '#87CEEB',  # 天蓝色
        ]
    }

def set_academic_style():
    """设置图表的全局学术风格"""
    colors = get_academic_colors()
    plt.style.use('seaborn-v0_8-paper')
    # --- 核心修正：提供一个备选字体列表，以增强兼容性 ---
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Heiti TC', 'Microsoft JhengHei', 'Arial Unicode MS']
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['figure.facecolor'] = colors['background']
    plt.rcParams['axes.facecolor'] = colors['background']
    plt.rcParams['text.color'] = colors['primary']
    plt.rcParams['axes.labelcolor'] = colors['primary']
    plt.rcParams['xtick.color'] = colors['secondary']
    plt.rcParams['ytick.color'] = colors['secondary']
    plt.rcParams['grid.color'] = colors['gridline']
    # --- 核心修改：再次增大全局基础字号 ---
    plt.rcParams['font.size'] = 22

In [3]:
# 残差分位数加速计算
import pandas as pd
import numpy as np
from pathlib import Path
from joblib import Parallel, delayed # 导入用于并行计算的库
import os # 导入os库以获取CPU核心数量

# =================================================================== #
#                       【1. 文件路径配置与模拟数据生成】             #
# =================================================================== #

# 请根据您的实际文件路径修改以下变量。
# 如果文件不存在，代码将自动创建模拟数据以确保可运行性。
RESIDUALS_FILE_3_1 = Path('E:/PBROE/ch3/pbroe3.1Residuals.csv')
RESIDUALS_FILE_4_1 = Path('E:/PBROE/ch4/pbroe4.1Residuals.csv')

# 输出目录设置为当前目录，如您所要求
OUTPUT_DIR = Path('.')
OUTPUT_DIR.mkdir(exist_ok=True) # 确保输出目录存在

# --- 为演示目的创建模拟残差数据文件 ---
# 在实际应用中，您将使用真实数据。如果您的真实文件已存在，此函数将不会执行创建操作。
def create_dummy_residuals_file(file_path, residual_col_name, num_stocks=200, num_months=120):
    """
    创建模拟残差数据文件。
    参数:
        file_path (Path): 要创建的模拟文件的路径。
        residual_col_name (str): 残差列的名称 ('residual_zscore' 或 'residual_zscore_adj')。
        num_stocks (int): 模拟的股票数量。
        num_months (int): 模拟的月份数量。
    """
    if not file_path.exists():
        print(f"检测到文件 '{file_path}' 不存在，正在创建模拟数据...")
        all_data = []
        start_date = pd.to_datetime('2010-01-31')
        for i in range(num_stocks):
            stkcd = str(600000 + i).zfill(6) # 模拟A股代码格式
            dates = pd.date_range(start=start_date, periods=num_months, freq='M')
            # 模拟残差值，使其具有一定的序列相关性，更接近真实数据分布
            residuals = np.random.randn(num_months).cumsum() / 5 + np.random.randn(num_months) * 0.2
            df_stock = pd.DataFrame({
                '调入日期': dates,
                'stkcd': stkcd,
                residual_col_name: residuals,
                'shortname': f'模拟公司{stkcd}',
                'indnme1': '模拟行业'
            })
            all_data.append(df_stock)
        pd.concat(all_data).to_csv(file_path, index=False)
        print(f"模拟文件 '{file_path}' 创建成功。")

# 调用函数创建模拟文件（如果不存在）
create_dummy_residuals_file(RESIDUALS_FILE_3_1, 'residual_zscore')
create_dummy_residuals_file(RESIDUALS_FILE_4_1, 'residual_zscore_adj')

# =================================================================== #
#                           【2. 数据加载与预处理】                   #
# =================================================================== #

def load_and_preprocess_residuals(file_path):
    """
    加载残差数据并进行预处理。
    参数:
        file_path (Path): 残差数据文件的路径。
    返回:
        pd.DataFrame: 加载并预处理后的DataFrame。
    """
    try:
        df = pd.read_csv(file_path)
        df['stkcd'] = df['stkcd'].astype(str).str.zfill(6)
        df['调入日期'] = pd.to_datetime(df['调入日期'])
        # 确保数据按股票代码和日期排序，这对于时序计算至关重要
        df = df.sort_values(by=['stkcd', '调入日期']).reset_index(drop=True)
        print(f"文件 '{file_path}' 加载成功，共 {len(df)} 条记录。")
        return df
    except FileNotFoundError:
        print(f"错误：未找到文件 '{file_path}'。请确保文件路径正确。")
        return pd.DataFrame()

df_residuals_3_1 = load_and_preprocess_residuals(RESIDUALS_FILE_3_1)
df_residuals_4_1 = load_and_preprocess_residuals(RESIDUALS_FILE_4_1)

# =================================================================== #
#                       【3. 时序残差分位数计算（并行版本）】         #
# =================================================================== #

def calculate_quantiles_for_single_stock(stock_df, residual_col, periods):
    """
    辅助函数：计算单个股票数据的时序残差分位数。
    此函数将在并行处理中被调用。
    参数:
        stock_df (pd.DataFrame): 单个股票的残差数据。
        residual_col (str): 残差列的名称。
        periods (list): 需要计算分位数的历史周期列表。
    返回:
        pd.DataFrame: 包含新增时序分位数列的单个股票DataFrame。
    """
    # 确保股票数据按日期排序，这对于滚动窗口计算至关重要
    stock_df = stock_df.sort_values(by='调入日期').copy() # 使用 .copy() 避免 SettingWithCopyWarning

    for period in periods:
        quantile_col_name = f'residual_quantile_{period}m'
        # 对残差列应用滚动计算
        # rank(pct=True) 计算百分位排名 (0-1之间)
        # iloc[-1] 获取当前（窗口中最后一个）元素的排名
        stock_df[quantile_col_name] = stock_df[residual_col].rolling(
            window=period, min_periods=1
        ).apply(lambda y: y.rank(pct=True).iloc[-1], raw=False)
    return stock_df

def calculate_time_series_quantiles(df, residual_col, periods=[10, 20, 50]):
    """
    计算个股时序残差分位数的主函数（并行版本）。
    将DataFrame按股票代码分组，并使用多核并行处理每个股票组。
    参数:
        df (pd.DataFrame): 包含残差数据（'stkcd', '调入日期', residual_col）的DataFrame。
        residual_col (str): 残差列的名称，例如 'residual_zscore' 或 'residual_zscore_adj'。
        periods (list): 需要计算分位数的历史周期列表（以月为单位）。
    返回:
        pd.DataFrame: 包含新增时序分位数列的完整DataFrame。
    """
    if df.empty:
        print("输入DataFrame为空，无法计算时序残差分位数。")
        return df

    # 检查指定的残差列是否存在于DataFrame中
    if residual_col not in df.columns:
        print(f"错误：DataFrame中未找到残差列 '{residual_col}'。请检查列名是否正确。")
        return df

    print(f"\n开始计算时序残差分位数（并行），针对列: '{residual_col}'...")

    # 获取CPU核心数量，用于并行计算
    num_cores = os.cpu_count()
    if num_cores is None:
        num_cores = 1 # 如果无法检测到核心数，则默认为单核
    print(f"检测到 {num_cores} 个CPU核心，将使用所有核心进行并行计算。")

    # 将DataFrame按股票代码分组，并转换为列表，以便 joblib 进行并行处理
    grouped_dfs = [group for _, group in df.groupby('stkcd')]

    # 使用 joblib.Parallel 进行并行计算
    # n_jobs=-1 表示使用所有可用的CPU核心
    results = Parallel(n_jobs=-1)(
        delayed(calculate_quantiles_for_single_stock)(stock_df, residual_col, periods)
        for stock_df in grouped_dfs
    )

    # 将所有并行计算的结果合并回一个DataFrame
    # 重新排序以保持原始顺序，并重置索引，确保最终DataFrame的结构一致性
    df_quantiles_parallel = pd.concat(results).sort_values(by=['stkcd', '调入日期']).reset_index(drop=True)

    print("时序残差分位数并行计算完成。")
    return df_quantiles_parallel

# =================================================================== #
#                       【4. 执行计算与保存结果】                     #
# =================================================================== #

# 对 pbroe3.1 残差数据计算时序分位数
# 注意：这里将 residual_col 设置为 'residual_zscore'，以匹配文件中的列名
if not df_residuals_3_1.empty:
    df_residuals_3_1_with_quantiles = calculate_time_series_quantiles(
        df_residuals_3_1.copy(), residual_col='residual_zscore'
    )
    output_path_3_1 = OUTPUT_DIR / 'pbroe3.1Residuals_with_quantiles.csv'
    df_residuals_3_1_with_quantiles.to_csv(output_path_3_1, index=False)
    print(f"\npbroe3.1 残差分位数数据已保存至: {output_path_3_1}")
else:
    print("pbroe3.1 残差数据加载失败，跳过时序分位数计算。")

print("\n" + "="*80 + "\n")

# 对 pbroe4.1 残差数据计算时序分位数
# 注意：这里将 residual_col 设置为 'residual_zscore_adj'，以匹配文件中的列名
if not df_residuals_4_1.empty:
    df_residuals_4_1_with_quantiles = calculate_time_series_quantiles(
        df_residuals_4_1.copy(), residual_col='residual_zscore_adj'
    )
    output_path_4_1 = OUTPUT_DIR / 'pbroe4.1Residuals_with_quantiles.csv'
    df_residuals_4_1_with_quantiles.to_csv(output_path_4_1, index=False)
    print(f"\npbroe4.1 残差分位数数据已保存至: {output_path_4_1}")
else:
    print("pbroe4.1 残差数据加载失败，跳过时序分位数计算。")


文件 'E:\PBROE\ch3\pbroe3.1Residuals.csv' 加载成功，共 551604 条记录。
文件 'E:\PBROE\ch4\pbroe4.1Residuals.csv' 加载成功，共 499191 条记录。

开始计算时序残差分位数（并行），针对列: 'residual_zscore'...
检测到 192 个CPU核心，将使用所有核心进行并行计算。
时序残差分位数并行计算完成。

pbroe3.1 残差分位数数据已保存至: pbroe3.1Residuals_with_quantiles.csv



开始计算时序残差分位数（并行），针对列: 'residual_zscore_adj'...
检测到 192 个CPU核心，将使用所有核心进行并行计算。
时序残差分位数并行计算完成。

pbroe4.1 残差分位数数据已保存至: pbroe4.1Residuals_with_quantiles.csv
